# TP1: Image Classification using Plant Leaf Dataset

This notebook implements an image classification solution using the 300_dataset, which contains images of different plant species organized in folders. Each folder represents a class (plant species).

## Objectives
- Load and preprocess the image dataset
- Extract features from the images
- Train a classification model
- Evaluate the model's performance
- Document the approach and results

## Dataset Structure
The dataset is organized in folders, with each folder representing a different plant species:
- Apta
- Indian Rubber Tree
- Karanj
- Kashid
- Nilgiri
- Pimpal
- Sita Ashok
- Sonmohar
- Vad
- Vilayati Chinch

In [ ]:
# Import necessary libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# PyTorch imports
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
from PIL import Image

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

ModuleNotFoundError: No module named 'tensorflow'

## 1. Data Loading and Exploration

First, let's explore the dataset to understand its structure and content.

In [ ]:
# Define the dataset path
dataset_path = "300_dataset"

# List all class folders
class_folders = os.listdir(dataset_path)
print(f"Total number of classes: {len(class_folders)}")
print(f"Class names: {class_folders}")

# Count images per class
class_counts = {}
for class_name in class_folders:
    class_path = os.path.join(dataset_path, class_name)
    if os.path.isdir(class_path):
        image_files = [f for f in os.listdir(class_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        class_counts[class_name] = len(image_files)

# Display class distribution
plt.figure(figsize=(12, 6))
sns.barplot(x=list(class_counts.keys()), y=list(class_counts.values()))
plt.xticks(rotation=90)
plt.title('Number of Images per Class')
plt.xlabel('Class')
plt.ylabel('Number of Images')
plt.tight_layout()
plt.show()

# Calculate total number of images
total_images = sum(class_counts.values())
print(f"Total number of images: {total_images}")

### Visualize Sample Images from Each Class

In [ ]:
# Display a sample image from each class
plt.figure(figsize=(15, 12))
for i, class_name in enumerate(class_counts.keys()):
    class_path = os.path.join(dataset_path, class_name)
    if os.path.isdir(class_path):
        image_files = [f for f in os.listdir(class_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        if image_files:
            # Get the first image file
            sample_image_path = os.path.join(class_path, image_files[0])
            # Read the image
            img = cv2.imread(sample_image_path)
            # Convert from BGR to RGB
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            # Display the image
            plt.subplot(4, 3, i+1)
            plt.imshow(img)
            plt.title(class_name)
            plt.axis('off')
            
plt.tight_layout()
plt.show()

### Check Image Dimensions

In [ ]:
# Check image dimensions to determine a standard size for preprocessing
image_dimensions = []
for class_name in class_counts.keys():
    class_path = os.path.join(dataset_path, class_name)
    if os.path.isdir(class_path):
        image_files = [f for f in os.listdir(class_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        if image_files:
            # Get a random sample of images (up to 5)
            sample_files = np.random.choice(image_files, min(5, len(image_files)), replace=False)
            for img_file in sample_files:
                img_path = os.path.join(class_path, img_file)
                img = cv2.imread(img_path)
                if img is not None:
                    image_dimensions.append(img.shape)

# Display some of the image dimensions
print("Sample of image dimensions (height, width, channels):")
for i, dim in enumerate(image_dimensions[:10]):
    print(f"Image {i+1}: {dim}")

# Calculate average dimensions
heights = [dim[0] for dim in image_dimensions]
widths = [dim[1] for dim in image_dimensions]
avg_height = int(np.mean(heights))
avg_width = int(np.mean(widths))

print(f"\nAverage height: {avg_height} pixels")
print(f"Average width: {avg_width} pixels")

# Define target image size for preprocessing
# We'll use a standard size to ensure uniform input to our model
target_size = (224, 224)  # Standard size for many CNN architectures
print(f"\nTarget image size for our model: {target_size}")

## 2. Data Preprocessing and Loading

Now we'll prepare the data for our model. We'll use Keras' ImageDataGenerator for data augmentation and loading.

In [ ]:
# Import necessary libraries
import os
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# Define image preprocessing parameters
target_size = (224, 224)
batch_size = 32

# Define transformations for training (with augmentation)
train_transforms = transforms.Compose([
    transforms.Resize(target_size),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Define transformations for validation (no augmentation)
val_transforms = transforms.Compose([
    transforms.Resize(target_size),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Custom dataset class for loading images from folders
class PlantLeafDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}
        
        self.image_paths = []
        self.labels = []
        
        # Collect all image paths and their labels
        for cls_name in self.classes:
            class_dir = os.path.join(root_dir, cls_name)
            for img_name in os.listdir(class_dir):
                if img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
                    img_path = os.path.join(class_dir, img_name)
                    self.image_paths.append(img_path)
                    self.labels.append(self.class_to_idx[cls_name])
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        
        # Open image with PIL
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Dataset path
dataset_path = "300_dataset"

# Create full dataset
full_dataset = PlantLeafDataset(dataset_path, transform=None)

# Count images per class
class_counts = {}
for cls_name in full_dataset.classes:
    class_dir = os.path.join(dataset_path, cls_name)
    if os.path.isdir(class_dir):
        image_files = [f for f in os.listdir(class_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        class_counts[cls_name] = len(image_files)

# Display class distribution
plt.figure(figsize=(12, 6))
sns.barplot(x=list(class_counts.keys()), y=list(class_counts.values()))
plt.xticks(rotation=90)
plt.title('Number of Images per Class')
plt.xlabel('Class')
plt.ylabel('Number of Images')
plt.tight_layout()
plt.show()

# Calculate total number of images
total_images = sum(class_counts.values())
print(f"Total number of images: {total_images}")

# Split dataset into train and validation sets
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])

# Apply transforms to the split datasets
class TransformedSubset(Dataset):
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform
        
    def __getitem__(self, idx):
        x, y = self.subset[idx]
        if self.transform:
            x = self.transform(x)
        return x, y
        
    def __len__(self):
        return len(self.subset)

train_dataset = TransformedSubset(train_dataset, train_transforms)
val_dataset = TransformedSubset(val_dataset, val_transforms)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# Print class information
class_indices = full_dataset.class_to_idx
class_names = {v: k for k, v in class_indices.items()}
print("Class indices:")
for class_name, idx in class_indices.items():
    print(f"{class_name}: {idx}")

### Visualize Augmented Images

Let's see how our data augmentation affects the images.

In [ ]:
# Get a batch of images and labels from the training loader
dataiter = iter(train_loader)
images, labels = next(dataiter)

# Convert from tensor to numpy for display
images_np = images.numpy()

# Function to unnormalize the images
def imshow(img):
    img = img.transpose((1, 2, 0))
    # Unnormalize
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    img = std * img + mean
    img = np.clip(img, 0, 1)
    return img

# Plot a few augmented images
plt.figure(figsize=(15, 10))
for i in range(min(9, images.shape[0])):
    plt.subplot(3, 3, i+1)
    plt.imshow(imshow(images_np[i]))
    plt.title(f'Class: {class_names[labels[i].item()]}')
    plt.axis('off')
plt.tight_layout()
plt.show()

## 3. Model Building

We'll build two models:
1. A simple CNN from scratch
2. A transfer learning model using VGG16 pretrained on ImageNet

### 3.1 Simple CNN Model

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define a simple CNN model in PyTorch
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        # First convolutional block
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        # Second convolutional block
        self.conv2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        # Third convolutional block
        self.conv3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        # Fully connected layers
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 28 * 28, 512),  # Adjusted based on input size
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.fc(x)
        return x

# Initialize the simple CNN model
num_classes = len(class_indices)
simple_cnn_model = SimpleCNN(num_classes).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(simple_cnn_model.parameters())

# Display model summary
print(simple_cnn_model)

### 3.2 Transfer Learning with VGG16

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

# Assuming device is already defined (e.g., device = torch.device("cuda" if torch.cuda.is_available() else "cpu"))

# Create a transfer learning model using VGG16 in PyTorch
def create_transfer_learning_model(num_classes):
    # Load pre-trained VGG16 model
    vgg16 = models.vgg16(pretrained=True)
    
    # Freeze the base model parameters
    for param in vgg16.parameters():
        param.requires_grad = False
    
    # Replace the classifier
    num_features = vgg16.classifier[6].in_features
    vgg16.classifier[6] = nn.Linear(num_features, num_classes)
    
    return vgg16

# Initialize the transfer learning model
transfer_model = create_transfer_learning_model(num_classes).to(device)

# Define loss function and optimizer
# Only optimize the classifier parameters that are not frozen
transfer_criterion = nn.CrossEntropyLoss()
transfer_optimizer = optim.Adam(filter(lambda p: p.requires_grad, transfer_model.parameters()))

# Display model summary
print(transfer_model)

## 4. Model Training and Evaluation

We'll train both models and compare their performance.

### 4.1 Train the Simple CNN Model

In [ ]:
# Training function for PyTorch model
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=25, patience=5):
    # Initialize history dictionary to store metrics
    history = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': []
    }
    
    # Initialize variables for early stopping
    best_val_loss = float('inf')
    patience_counter = 0
    best_model_state = None
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]"):
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            # Backward pass and optimize
            loss.backward()
            optimizer.step()
            
            # Statistics
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        epoch_train_loss = running_loss / len(train_loader.dataset)
        epoch_train_acc = correct / total
        
        # Validation phase
        model.eval()
        running_loss = 0.0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for inputs, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Val]"):
                inputs, labels = inputs.to(device), labels.to(device)
                
                # Forward pass
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                # Statistics
                running_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        epoch_val_loss = running_loss / len(val_loader.dataset)
        epoch_val_acc = correct / total
        
        # Store metrics
        history['train_loss'].append(epoch_train_loss)
        history['train_acc'].append(epoch_train_acc)
        history['val_loss'].append(epoch_val_loss)
        history['val_acc'].append(epoch_val_acc)
        
        # Print epoch summary
        print(f"Epoch {epoch+1}/{num_epochs} - "
              f"Train Loss: {epoch_train_loss:.4f}, Train Acc: {epoch_train_acc:.4f} - "
              f"Val Loss: {epoch_val_loss:.4f}, Val Acc: {epoch_val_acc:.4f}")
        
        # Early stopping
        if epoch_val_loss < best_val_loss:
            best_val_loss = epoch_val_loss
            patience_counter = 0
            best_model_state = model.state_dict().copy()
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break
    
    # Load best model state
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
    
    return model, history

# Train the simple CNN model
simple_cnn_model, history_simple = train_model(
    simple_cnn_model, 
    criterion, 
    optimizer, 
    train_loader, 
    val_loader,
    num_epochs=25,
    patience=5
)

### 4.2 Train the Transfer Learning Model

In [ ]:
# Train the transfer learning model
transfer_model, history_transfer = train_model(
    transfer_model, 
    transfer_criterion, 
    transfer_optimizer, 
    train_loader, 
    val_loader,
    num_epochs=15,  # Transfer learning usually requires fewer epochs
    patience=5
)

### 4.3 Plot Training History for Both Models

In [ ]:
# Plot training history for both models
def plot_training_history(history1, history2, title1, title2):
    plt.figure(figsize=(15, 5))
    
    # Plot accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history1['train_acc'], label=f'{title1} - Training')
    plt.plot(history1['val_acc'], label=f'{title1} - Validation')
    plt.plot(history2['train_acc'], label=f'{title2} - Training')
    plt.plot(history2['val_acc'], label=f'{title2} - Validation')
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend()
    
    # Plot loss
    plt.subplot(1, 2, 2)
    plt.plot(history1['train_loss'], label=f'{title1} - Training')
    plt.plot(history1['val_loss'], label=f'{title1} - Validation')
    plt.plot(history2['train_loss'], label=f'{title2} - Training')
    plt.plot(history2['val_loss'], label=f'{title2} - Validation')
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

# Plot the training history
plot_training_history(history_simple, history_transfer, 'Simple CNN', 'Transfer Learning')

### 4.4 Evaluate Both Models on Validation Data

In [ ]:
# Function to evaluate model
def evaluate_model(model, data_loader):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    criterion = nn.CrossEntropyLoss()
    
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            # Statistics
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    eval_loss = running_loss / len(data_loader.dataset)
    eval_acc = correct / total
    
    return eval_loss, eval_acc

# Evaluate the simple CNN model
simple_cnn_evaluation = evaluate_model(simple_cnn_model, val_loader)
print(f"Simple CNN - Validation Loss: {simple_cnn_evaluation[0]:.4f}, Validation Accuracy: {simple_cnn_evaluation[1]:.4f}")

# Evaluate the transfer learning model
transfer_evaluation = evaluate_model(transfer_model, val_loader)
print(f"Transfer Learning - Validation Loss: {transfer_evaluation[0]:.4f}, Validation Accuracy: {transfer_evaluation[1]:.4f}")

### 4.5 Generate Predictions and Confusion Matrix

Let's use the better performing model to generate predictions and visualize the confusion matrix.

In [ ]:
# Choose the better performing model (usually the transfer learning model)
better_model = transfer_model if transfer_evaluation[1] > simple_cnn_evaluation[1] else simple_cnn_model
model_name = "Transfer Learning" if transfer_evaluation[1] > simple_cnn_evaluation[1] else "Simple CNN"

# Get predictions and true labels
all_preds = []
all_labels = []

better_model.eval()
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = better_model(inputs)
        _, preds = torch.max(outputs, 1)
        
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Convert to numpy arrays
y_pred = np.array(all_preds)
y_true = np.array(all_labels)

# Generate classification report
print(f"Classification Report for {model_name} Model:")
class_names_list = [class_names[i] for i in range(len(class_names))]
print(classification_report(y_true, y_pred, target_names=class_names_list))

# Generate confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)

# Plot confusion matrix
plt.figure(figsize=(12, 10))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names_list, 
            yticklabels=class_names_list)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title(f'Confusion Matrix - {model_name} Model')
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

## 5. Visualize Predictions

Let's visualize some predictions from the validation set.

In [ ]:
# Reset the validation generator
validation_generator.reset()

# Get a batch of validation images and labels
dataiter = iter(val_loader)
X_val, y_val = next(dataiter)

# Move to device
X_val = X_val.to(device)

# Make predictions
better_model.eval()
with torch.no_grad():
    predictions = better_model(X_val)
    predictions = torch.softmax(predictions, dim=1)

# Convert tensors to numpy for visualization
X_val_np = X_val.cpu().numpy()
y_val_np = y_val.cpu().numpy()
predictions_np = predictions.cpu().numpy()

# Plot images with true and predicted labels
plt.figure(figsize=(15, 10))
for i in range(min(9, len(X_val))):
    plt.subplot(3, 3, i+1)
    plt.imshow(imshow(X_val_np[i]))
    
    true_class_idx = y_val_np[i]
    pred_class_idx = np.argmax(predictions_np[i])
    
    true_class = class_names[true_class_idx]
    pred_class = class_names[pred_class_idx]
    confidence = predictions_np[i][pred_class_idx] * 100
    
    title_color = 'green' if true_class_idx == pred_class_idx else 'red'
    plt.title(f'True: {true_class}\nPred: {pred_class}\nConf: {confidence:.1f}%', 
              color=title_color)
    plt.axis('off')
    
plt.tight_layout()
plt.show()

## 6. Model Saving

Let's save the better performing model for future use.

In [ ]:
# Save the better model
model_save_path = f"{model_name.replace(' ', '_').lower()}_model.pt"
torch.save(better_model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

# Save class indices for future reference
import json
with open('class_indices.json', 'w') as f:
    json.dump(class_indices, f)
print("Class indices saved to class_indices.json")

## 7. Create a Function for Making Predictions on New Images

In [ ]:
def predict_image(model, image_path, class_names, target_size=(224, 224)):
    """
    Predict the class of a new image using PyTorch.
    
    Args:
        model: Trained PyTorch model
        image_path: Path to the image file
        class_names: Dictionary mapping class indices to class names
        target_size: Size to resize the image to
        
    Returns:
        Predicted class name and confidence
    """
    # Set model to evaluation mode
    model.eval()
    
    # Load and preprocess the image
    img = Image.open(image_path).convert('RGB')
    
    # Apply the same transformations used for validation
    transform = transforms.Compose([
        transforms.Resize(target_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    
    img_tensor = transform(img).unsqueeze(0).to(device)  # Add batch dimension
    
    # Make prediction
    with torch.no_grad():
        outputs = model(img_tensor)
        probabilities = torch.softmax(outputs, dim=1)
        confidence, predicted_idx = torch.max(probabilities, 1)
    
    # Convert to Python values
    pred_class_idx = predicted_idx.item()
    confidence_value = confidence.item() * 100
    
    # Get class name
    pred_class_name = class_names[pred_class_idx]
    
    return pred_class_name, confidence_value

# Test the prediction function on a sample image
# Find a sample image
sample_class = list(class_counts.keys())[0]  # Get the first class
sample_class_path = os.path.join(dataset_path, sample_class)
if os.path.isdir(sample_class_path):
    image_files = [f for f in os.listdir(sample_class_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    if image_files:
        sample_image_path = os.path.join(sample_class_path, image_files[0])
        
        # Predict
        pred_class, confidence = predict_image(better_model, sample_image_path, class_names)
        
        # Display the image and prediction
        img = cv2.imread(sample_image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        plt.figure(figsize=(8, 6))
        plt.imshow(img)
        plt.title(f"Predicted: {pred_class} (Confidence: {confidence:.1f}%)\nTrue: {sample_class}")
        plt.axis('off')
        plt.show()
        
        print(f"Predicted class: {pred_class}")
        print(f"Confidence: {confidence:.1f}%")
        print(f"True class: {sample_class}")

## 8. Conclusion

In this notebook, we have:

1. Loaded and explored the 300_dataset of plant species images
2. Preprocessed the images, including resizing and data augmentation
3. Built two models:
   - A simple CNN from scratch
   - A transfer learning model using VGG16
4. Trained and evaluated both models
5. Visualized the results, including confusion matrices and example predictions
6. Saved the better-performing model for future use
7. Created a utility function for making predictions on new images

### Key Findings:

- The transfer learning model generally performs better than the simple CNN model, which is expected since it leverages pre-trained weights on a large dataset (ImageNet).
- Data augmentation helps to improve model generalization, especially when working with a limited dataset.
- The confusion matrix reveals which classes are more difficult to distinguish, providing insights for potential model improvements.

### Potential Improvements:

1. Fine-tune the pre-trained layers of the VGG16 model
2. Try other pre-trained architectures like ResNet, Inception, or EfficientNet
3. Increase dataset size through more aggressive data augmentation
4. Implement ensemble methods by combining predictions from multiple models
5. Apply techniques to handle class imbalance if present in the dataset